In [25]:
import pandas as pd
import time
import board
import neopixel



ModuleNotFoundError: No module named 'board'

In [26]:
# NeoPixel LED Configuration
#LED_PIN			= board.D18		# GPIO pin connected to the pixels (18 is PCM).
#PIXEL_ORDER		= neopixel.GRB		# Strip type and colour ordering
LED_PIN = 1
PIXEL_ORDER = 'GRB'

COLOUR_DICT = {
    'VFR'  : (255,0,0),
    'MVFR' : (0,0,255),
    'IFR'  : (0,255,0),
    'LIFR' : (0,125,125),
    'LIGHTNING' : (255,255,255),
    'HIGH_WINDS' : (255,255,0),
    'OFF' : (0,0,0)
}

WIND_KTS = 15
HIGH_WIND_KTS = 25

LOOP_TIME = 300 # time in seconds (roughly) for LED loop to run


In [27]:
df = pd.read_csv('https://www.aviationweather.gov/adds/dataserver_current/current/metars.cache.csv.gz', compression='gzip', skiprows=5)

In [28]:
df.to_csv('metars.csv')

In [10]:
len(df)

4845

In [16]:
"2022-11-26T20:33:00Z"[11:16]

'20:33'

In [24]:
import math
math.floor(float("200.0"))

200

In [12]:
display = 0
display += 1
type(display)

int

In [9]:
df.iloc[0]['raw_text']

'YSNF 262033Z AUTO /////KT 5000 +RA ////// 19/17 Q1010 RF//./////./'

In [4]:
df.head()

,raw_text,station_id,observation_time,latitude,longitude,temp_c,dewpoint_c,wind_dir_degrees,wind_speed_kt,wind_gust_kt,...,maxT24hr_c,minT24hr_c,precip_in,pcp3hr_in,pcp6hr_in,pcp24hr_in,snow_in,vert_vis_ft,metar_type,elevation_m
0,YSNF 262033Z AUTO /////KT 5000 +RA ////// 19/1...,YSNF,2022-11-26T20:33:00Z,-29.03,167.93,19.0,17.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,113.0
1,UMMS 262033Z 34001MPS 2400 0400E FZFG OVC002 M...,UMMS,2022-11-26T20:33:00Z,53.88,28.03,-4.0,-4.0,340.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,196.0
2,CYIV 262033Z AUTO 28010G16KT 2 1/4SM -SN OVC00...,CYIV,2022-11-26T20:33:00Z,53.85,-94.65,-2.0,-3.0,280.0,10.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,234.0
3,CYGV 262033Z AUTO 09005KT 6SM BR OVC004 01/01 ...,CYGV,2022-11-26T20:33:00Z,50.28,-63.62,1.0,1.0,90.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,32.0
4,KHAI 262031Z AUTO 18010KT 10SM CLR 11/M02 A299...,KHAI,2022-11-26T20:31:00Z,41.97,-85.60,11.0,-2.0,180.0,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,METAR,250.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4845 entries, 0 to 4844
Data columns (total 44 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   raw_text                       4845 non-null   object 
 1   station_id                     4845 non-null   object 
 2   observation_time               4845 non-null   object 
 3   latitude                       4793 non-null   float64
 4   longitude                      4793 non-null   float64
 5   temp_c                         4794 non-null   float64
 6   dewpoint_c                     4777 non-null   float64
 7   wind_dir_degrees               4771 non-null   float64
 8   wind_speed_kt                  4771 non-null   float64
 9   wind_gust_kt                   677 non-null    float64
 10  visibility_statute_mi          4386 non-null   float64
 11  altim_in_hg                    4517 non-null   float64
 12  sea_level_pressure_mb          1594 non-null   f

In [60]:
file = open('airports.txt', 'r')

with file as f:
    airports = [line.rstrip() for line in f]

file.close()


In [6]:
airports

['EHPG',
 'EHKV',
 'EHFS',
 'EHWO',
 'EBBR',
 'EBAW',
 'EBDT',
 'EBBL',
 'EHEH',
 'EHGG',
 'EHDL',
 'EHLW',
 'EHMA',
 'EHMG',
 'EHKD',
 'EHAM',
 'EHLE',
 'EHVL']

In [7]:
len(airports)

18

In [61]:
airport_metar_df = df[df['station_id'].isin(airports)][['station_id', 'observation_time', 'temp_c', 'dewpoint_c', 'wind_dir_degrees', 'wind_speed_kt', 'wind_gust_kt', 'sea_level_pressure_mb', 'wx_string', 'flight_category']].copy().reset_index()

#airport_metar_df['flight_category'] = airport_metar_df['flight_category'].astype('string')

In [62]:
airport_metar_df.head(18)


,index,station_id,observation_time,temp_c,dewpoint_c,wind_dir_degrees,wind_speed_kt,wind_gust_kt,sea_level_pressure_mb,wx_string,flight_category
0,122,EHWO,2022-11-22T16:39:00Z,9.0,6.0,200.0,7.0,NaN,NaN,NaN,MVFR
1,1779,EHEH,2022-11-22T16:30:00Z,8.0,6.0,170.0,8.0,NaN,NaN,NaN,MVFR
2,1820,EHVL,2022-11-22T16:25:00Z,7.0,5.0,150.0,18.0,NaN,NaN,NaN,MVFR
3,1826,EHPG,2022-11-22T16:25:00Z,11.0,5.0,220.0,24.0,NaN,NaN,NaN,VFR
4,1827,EHMG,2022-11-22T16:25:00Z,8.0,5.0,160.0,26.0,NaN,NaN,NaN,MVFR
5,1828,EHMA,2022-11-22T16:25:00Z,7.0,4.0,130.0,20.0,NaN,NaN,NaN,MVFR
6,1829,EHLW,2022-11-22T16:25:00Z,7.0,4.0,140.0,11.0,NaN,NaN,NaN,VFR
7,1830,EHLE,2022-11-22T16:25:00Z,7.0,5.0,160.0,12.0,NaN,NaN,NaN,MVFR
8,1831,EHKV,2022-11-22T16:25:00Z,10.0,7.0,170.0,24.0,NaN,NaN,NaN,VFR
9,1832,EHKD,2022-11-22T16:25:00Z,7.0,4.0,150.0,10.0,NaN,NaN,NaN,MVFR


In [9]:
df[df['station_id'].isin(airports)].columns

Index(['raw_text', 'station_id', 'observation_time', 'latitude', 'longitude',
       'temp_c', 'dewpoint_c', 'wind_dir_degrees', 'wind_speed_kt',
       'wind_gust_kt', 'visibility_statute_mi', 'altim_in_hg',
       'sea_level_pressure_mb', 'corrected', 'auto', 'auto_station',
       'maintenance_indicator_on', 'no_signal', 'lightning_sensor_off',
       'freezing_rain_sensor_off', 'present_weather_sensor_off', 'wx_string',
       'sky_cover', 'cloud_base_ft_agl', 'sky_cover.1', 'cloud_base_ft_agl.1',
       'sky_cover.2', 'cloud_base_ft_agl.2', 'sky_cover.3',
       'cloud_base_ft_agl.3', 'flight_category',
       'three_hr_pressure_tendency_mb', 'maxT_c', 'minT_c', 'maxT24hr_c',
       'minT24hr_c', 'precip_in', 'pcp3hr_in', 'pcp6hr_in', 'pcp24hr_in',
       'snow_in', 'vert_vis_ft', 'metar_type', 'elevation_m'],
      dtype='object')

In [30]:
airport_metar_df['wx_string'] = airport_metar_df['wx_string'].astype('string')

In [45]:
airport_metar_df['wx_string'].isnull()[1]


True

In [53]:
# Need to flash on/off or yellow during wind cycle
wind_cycle = False
loop_time = LOOP_TIME

#pixels = neopixel.NeoPixel(LED_PIN, len(airports), brightness = 0.2, pixel_order = PIXEL_ORDER, auto_write = False)


while loop_time > 0:
    # keep track of LED number
    i = 0
    for a in airports:
        colour = COLOUR_DICT['OFF']
        a_metar = airport_metar_df.loc[airport_metar_df.station_id == a].copy()
        #print(a_metar)
        if len(a_metar) == 1:
            # Work out correct colour
            flight_category = a_metar.flight_category.values[0]
            if flight_category in COLOUR_DICT.keys() :
                colour = COLOUR_DICT[flight_category]

            if wind_cycle:
                # Adjust colours only on wind cycle AND if it is windy!
                print(a_metar.wx_string.isnull().values[0])
                lightning_list = ['TS', 'LTG', 'VCTS']
                if not a_metar.wx_string.isnull().values[0] :
                    if any(x in a_metar.wx_string.values[0] for x in lightning_list) :
                        print('Lightning')
                        colour = COLOUR_DICT['LIGHTNING']
                elif a_metar.wind_gust_kt.values[0] != None :
                    colour = COLOUR_DICT['HIGH_WINDS']
                elif not a_metar.wind_speed_kt.isnull().values[0] :
                    if a_metar.wind_speed_kt.values[0] > HIGH_WIND_KTS :
                        colour = COLOUR_DICT['HIGH_WINDS']
                
        print(f"Airport {a} and flight_category is {flight_category}. Setting LED {i} for {a} to {colour}")

        ##pixels[i] = colour
        i += 1

    time.sleep(2)
    wind_cycle = False if wind_cycle else True
    loop_time -= 1


Airport EHPG and flight_category is VFR. Setting LED 0 for EHPG to (255, 0, 0)
Airport EHKV and flight_category is MVFR. Setting LED 1 for EHKV to (0, 0, 255)
Airport EHFS and flight_category is VFR. Setting LED 2 for EHFS to (255, 0, 0)
Airport EHWO and flight_category is MVFR. Setting LED 3 for EHWO to (0, 0, 255)
Airport EBBR and flight_category is VFR. Setting LED 4 for EBBR to (255, 0, 0)
Airport EBAW and flight_category is MVFR. Setting LED 5 for EBAW to (0, 0, 255)
Airport EBDT and flight_category is MVFR. Setting LED 6 for EBDT to (0, 0, 255)
Airport EBBL and flight_category is MVFR. Setting LED 7 for EBBL to (0, 0, 255)
Airport EHEH and flight_category is MVFR. Setting LED 8 for EHEH to (0, 0, 255)
Airport EHGG and flight_category is VFR. Setting LED 9 for EHGG to (255, 0, 0)
Airport EHDL and flight_category is MVFR. Setting LED 10 for EHDL to (0, 0, 255)
Airport EHLW and flight_category is VFR. Setting LED 11 for EHLW to (255, 0, 0)
Airport EHMA and flight_category is MVFR. S

KeyboardInterrupt: 

In [15]:
len(airport_metar_df[airport_metar_df.station_id == 'EBBL'])

1

In [26]:

COLOUR_DICT[None]

KeyError: None

In [25]:
'VFR' in COLOUR_DICT.keys()

True

In [14]:


theval = "LTG RA"
lightning_list = ['TS', 'LTG', 'VCTS']
if theval != None :
    if any(x in theval for x in lightning_list) :
        print('Lightning')
        colour = COLOUR_DICT['LIGHTNING']



Lightning


In [ ]:
for i in range(len(airports)):
    pixels[i] = COLOUR_DICT['OFF']
pixels.show()